# A Brief Interlude: On Coupling and Abstractions

* What makes a good abstraction?
* What do we want from abstractions?
* How do they relate to testing?
* Locally, coupling is good. Different components are supporting each other.
* Globally, coupling is bad. It increases the cost to change our code.

## Abstracting State Aids Testability

* Let's write code for synchornising two file directories *source* and *dest*
    1. If a file exists in source, but not in dest, copy it over
    1. If a file exists in source, but has a different name than in dest, rename the dest file to match
    1. If a file exists in dest, but not in source, delete it.
    
* (1) and (3) are straightforward. We can just compare two lists of the paths.
* (2) is more difficult. To detect renames, we'll use a hashing function.

In [1]:
import hashlib

BLOCKSIZE = 65536
def hash_file(path):
    hasher = hashlib.sha1()
    with path.open("rb") as file:
        buf = file.read(BLOCKSIZE)
        while buf:
            hasher.update(buf)
            buf = file.read(BLOCKSIZE)
    return hasher.hexdigest()

* Now to write the code that makes decisions - the business logic.
* Start simple and iteratively make the solution better
* Let's start with a basic sync algorithm

In [3]:
import os
import shutil
from pathlib import Path

def sync_simple(source, dest):
    # Walk source and build a dict of filenames and hashes
    source_hashes = {}
    for folder, _, files in os.walk(source):
        for filename in files:
            source_hashs[hash_file(Path(folder) / filename)] = filename
            
    seen = set()  # track files in dest
    
    # Walk dest and get filenames and hashes
    for folder, _, files in os.walk(dest):
        for filename in files:
            dest_path = Path(folder) / filename
            dest_hash = hash_file(dest_path)
            seen.add(dest_hash)
            
            # Delete files in dest not in source
            if dest_hash not in source_hashes:
                dest_path.remove()
                
            # Change name if doesn't match source
            elif dest_hash in source_hashes and filename != source_hashes[dest_hash]:
                shutil.move(dest_path, Path(folder) / source_hashes[dest_hash])
            
    # Copy files in source not in dest to dest
    for src_hash, filename in source_hashes.items():
        if src_hash not in seen:
            shutil.copy(Path(source) / filename, Path(dest) / filename)

* Let's try writing some tests now

In [5]:
import tempfile
from pathlib import Path
import shutil

def test_when_file_exists_in_the_source_but_not_in_dest():
    try:
        source = tempfile.mkdtemp()
        dest = tempfile.mkdtemp()
        
        content = "I am a file"
        (Path(source) / "my-file").write_text(content)
        
        sync_simple(source, dest)
        
        expected_path = Path(dest) / "my-file"
        assert expected_path.exists()
        assert expected_path.read_text() == content
        
    finally:
        shutil.rmtree(source)
        shutil.rmtree(dest)
        
def test_when_a_file_has_been_renamed_in_the_source():
    try:
        source = tempfile.mkdtemp()
        dest = tempfile.mkdtemp()
        
        content = "I am a file"
        source_path = Path(source) / "src-file"
        dest_path = Path(dest) / "dest-file"
        synced_dest_path = Path(dest) / "src-file"
        source_path.write_text(content)
        dest_path.write_text(content)
        
        sync_simple(source, dest)
        
        assert not dest_path.exists()
        assert synced_dest_path.read_text() == content
        
    finally:
        shutil.rmtree(source)
        shutil.rmtree(dest)

* This is a lot of setup for two pretty straightforward cases.
* Here, our domain logic (figure out the difference between two directories) has been tightly coupled to I/O.
* Our code is also buggy (we've hidden some bugs amongst the implementation). These will require writing even further tests
* Our code is not very extensible. E.g., if we want to implement a `--dry-run` flag that gets our code to print out what it would do, we couldn't do so.

## Choosing the Right Abstraction(s)

* Our code is doing three things.
    1. Interrogate our filesystem using `os.walk` and determine hashes
    1. Decide whether a file is new, renamed, or redundant
    1. Copy, move, or delete files to match the source
* These are our code's *resposibilities*
* We want to find a simplifying abstraction for each responsibility
* For (1) to (2), we've already started to abstract by using a dictionary of hashes to paths. We could just compare two dicts. This would abstract the current state of the filesystem.
* For abstracting (2) to (3), we'll separate what we want from how to do it. Our program will output a list of commands that look like

```python
("COPY", "sourcepath", "destpath")
("MOVE", "old", "new")
```

* Now our tests would expect two filesystem dicts as inputs and would expect list of tuples as outputs.
* Instead of "Given this actual filesystem, when I run my function, check what actions have happened", we say, "Given this abstraction of a filesystem, what abstraction of a filesystem actions will happen?"

In [6]:
def test_when_file_in_source_but_not_in_dest():
    src_hashes = {"hash1": "filename1"}
    dst_hashes = {}
    expected_actions = [("COPY", "/src/filename1", "/dst/filename1")]
    
    # TODO
    
def test_when_file_renamed():
    src_hashes = {"hash1": "filename1"}
    dst_hashes = {"hash1": "filename2"}
    expected_actions = [("MOVE", "/dst/filename2", "/dst/filename1")]
    
    # TODO

## Implementing Our Chosen Abstractions

* How do we write the above tests and change our implementation?
* Goal: isolate the clever part of our system and test it without needing a real filesystem.
* *Functional Core, Imperative Shell (FCIS)* - Create a "core" of code without dependencies on external state and see how it responds when we give it input from the outside world.
    * Stateful part of code should be separate from logic
    * Top-level function will contain almost no logic, it will just be a collection of steps
        1. Gather inputs
        1. Call logic
        1. Apply outputs

In [1]:
def sync_fcis(source, dest):
    # Imperative shell step 1: gather inputs
    source_hashes = paths_and_hashes(source)
    dest_hashes = paths_and_hashes(dest)
    
    # Functional core step 2: call logic
    actions = actions(source_hashes, dest_hashes, source, dest)
    
    # Imperative shell step 3: apply outputs
    for action, *paths in actions:
        if action == "COPY":
            shutil.copyfile(*paths)
        if action == "MOVE":
            shutil.move(*paths)
        if action == "DELETE":
            os.remove(paths[0])
            
# Does only I/O
def paths_and_hashes(root):
    hashes = {}
    for folder, _, files in os.walk(root):
        for filename in files:
            hashes[hash_file(Path(folder) / filename)] = filename
    return hashes

def actions(src_hashes, dst_hashes, src_folder, dst_folder):
    for sha, filename in src_hashes.items():
        if sha not in dst_hashes:
            sourcepath = Path(src_folder) / filename
            destpath = Path(dst_folder) / filename
            yield "COPY", sourcepath, destpath
        
        elif dst_hashes[sha] != filename:
            olddestpath = Path(dst_folder) / dst_hashes[sha]
            newdestpath = Path(dst_folder) / filename
            yield "MOVE", olddestpath, newdestpath
            
    for sha, filename in dst_hashes.items():
        if sha not in src_hashes:
            yield "DELETE", dst_folder / filename

* Now our tests look like

In [2]:
def test_when_file_in_source_but_not_in_dest():
    src_hashes = {"hash1": "filename1"}
    dst_hashes = {}
    expected_actions = [("COPY", "/src/filename1", "/dst/filename1")]
    actions = actions(src_hashes, dst_hashes, Path("/src"), Path("/dest"))
    assert list(actions) == expected_actions
    
def test_when_file_renamed():
    src_hashes = {"hash1": "filename1"}
    dst_hashes = {"hash1": "filename2"}
    expected_actions = [("MOVE", "/dst/filename2", "/dst/filename1")]
    actions = actions(src_hashes, dst_hashes, Path("/src"), Path("/dest"))
    assert list(actions) == expected_actions

* Our tests easily check the core of our code without getting bogged down in I/O
* How should we now test sync?
    * Ignore testing sync because the actual functionality of our code is tied up in the logic, not in I/O
    * Do some integration/acceptance tests for sync
    * Modify sync so that it can be unit-tested and end-to-end tested.
        * Uncle Bob calls this approach *edge-to-edge testing*
        
### Testing Edge-To-Edge with Fakes and Dependency Injection

* Testing that invoke a whole system, but fake the I/O

In [4]:
def sync_edge_to_edge(reader, filesystem, source_root, dest_root):
    source_hashes = reader(source_root)
    dest_hashes = reader(dest_root)
    
    for sha, filename in source_hashes.items():
        if sha not in dest_hashes:
            sourcepath = source_root / filename
            destpath = dest_root / filename
            filesystem.copy(sourcepath, destpath)
            
        elif dest_hashes[sha] != filename:
            olddestpath = dest_root / dest_hashes[sha]
            newdestpath = dest_root / filename
            filesystem.move(olddestpath, newdestpath)
    
    for sha, filename in dest_hashes.items():
        if sha not in source_hashes:
            filesystem.delete(dest_root / filename)
            
            
# Testing
class FakeFileSystem(list):
    
    def copy(self, src, dest):
        self.append(("COPY", src, dest))
        
    def move(self, src, dest):
        self.append(("MOVE", src, dest))
        
    def delete(self, dest):
        self.append(("DELETE", dest))

* We can use `FakeFileSystem` to test the implementation now.
    * Using `list` to build tests allows us to use asserts like `assert "foo" in my_list`
    * `FakeFileSystem` just appends to itself for later inspection. This is an example of a *spy* object.
* Pro: Test acts on production code
* Con: Stateful components are made explicit and are passed around
* This pattern is called *testing-induced design damage*. The design is made worse to improve testing
    * Designing for testability means designing for extensibility.
    * Trade off complexity for a cleaner design that admits novel use cases

### Why not patch?

* We could use `mock.patch`
    * Patching allows for unit testing, but it doesn't improve the design
    * Tests that use mock tend to be more coupled to implementation
    * Mock leads to complicated test suites that don't explain the code
* This is a difference between the [London school and Chicago (classical) school of TDD](https://softwareengineering.stackexchange.com/questions/123627/what-are-the-london-and-chicago-schools-of-tdd). This book follows the patterns in the classical school.

## Wrap Up

* Finding the right abstraction is tricky. Here are some questions to consider.
    * Can I choose a familiar Python data structure to represent the state and then try to imagine a single function that can return that state?
    * Where can I draw a line between my systems? Where can I carve out a [seam](https://www.informit.com/articles/article.aspx?p=359417&seqNum=2) to stick that abstraction in?
    * What is a sensible way to divide things into components with responsibilities?
    * What implicit concepts can I make explicit?
    * What are the dependencies and what is the core business logic?